In [39]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup
import requests

with open('List of postal codes of Canada.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [22]:

Postal_code=[]
Borough=[]
Neighbourhood=[]
for row in soup.find('tbody').find_all('tr'):
    cells = row.find_all('td')
    Postal_code.append(cells[0].text.rstrip('\n'))
    Borough.append(cells[1].text.rstrip('\n'))
    try:
        Neighbourhood.append(cells[2].text.rstrip('\n'))
    except:
        Neighbourhood.append('Not Assigned')



Constructing the data frame

In [23]:
import pandas as pd
df = pd.DataFrame({'PostalCode': Postal_code,
                  'Borough': Borough,
                  'Neighborhood': Neighbourhood})
df.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Removing "Not assigned" Boroughs

In [26]:
df1=df[df.Borough !='Not assigned'].reset_index(drop=True)
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [27]:
df2= df1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [36]:
for index,row in df2.iterrows():
    if row["Neighborhood"] == "":
        row["Neighborhood"] = row["Borough"]
        
df2


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [38]:
df2.shape

(103, 3)